In [72]:
import pandas as pd
import pickle
from collections import Counter



#model building
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix


#roc building

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle




import seaborn as sns

pd.set_option('display.max_rows', 300)

survey = pd.read_csv('/Users/williamyee/Documents/Data Bootcamp/Classification-Project-DSjobs/Data/kaggle-survey-2017/multi_r_.csv', encoding = 'ISO-8859-1')

list(survey.columns)

survey.JobSatisfaction.value_counts()



survey.head()

survey.EmploymentStatus.unique()

survey.groupby('EmploymentStatus').count()

survey.UniversityImportance.value_counts()

survey.JobSatisfaction.value_counts()




/Users/williamyee/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (32,84,87,88,100,110,117,124,125,128,130,131,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


7                          1448
8                          1427
6                           765
9                           677
5                           627
10 - Highly Satisfied       589
3                           358
4                           354
1 - Highly Dissatisfied     167
I prefer not to share       148
2                           117
Name: JobSatisfaction, dtype: int64

In [85]:
conditions = [  survey.JobSatisfaction.isin(['7','8','9','10 - Highly Satisfied']),
                survey.JobSatisfaction.isin(['1 - Highly Dissatisfied','2','3','4','5','6','I prefer not to share'])
             ]
choices = ['Satisfied','Not Satisfied']
survey['JobSatisfaction'] = np.select(conditions, choices, default=survey.JobSatisfaction)



In [86]:
survey.JobSatisfaction.value_counts()

survey.TitleFit.value_counts()

survey.GenderSelect.value_counts()

survey.EmploymentStatus.value_counts()

survey.CareerSwitcher.value_counts()

survey.CurrentEmployerType.value_counts()

survey.Tenure.value_counts()

survey.LearningDataScienceTime.value_counts()

survey.CurrentJobTitleSelect.value_counts()

survey.StudentStatus.value_counts()

survey = survey[survey.JobSatisfaction.notna()]
survey.shape

percent_missing = survey.isnull().sum()*100/len(survey)
missing_value_df = pd.DataFrame({'columnname':survey.columns, 'percent_missing':percent_missing})
seventy_five_data = missing_value_df[missing_value_df.percent_missing<40]

seventy_five_data

survey_minimal_missing= survey[list(seventy_five_data.columnname)]
X = survey_minimal_missing.drop(columns = 'JobSatisfaction')
y = survey_minimal_missing['JobSatisfaction']
X

,Unnamed: 0,GenderSelect,Country,Age,EmploymentStatus,CodeWriter,CurrentJobTitleSelect,TitleFit,CurrentEmployerType,MLToolNextYearSelect,...,WorkDataVisualizations,WorkInternalVsExternalTools,WorkMLTeamSeatSelect,WorkDataStorage,WorkDataSharing,WorkCodeSharing,RemoteWork,CompensationAmount,CompensationCurrency,SalaryChange
0,1,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,Yes,DBA/Database Engineer,Fine,Employed by a company that doesn't perform adv...,SAS Base,...,26-50% of projects,Do not know,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,"Company Developed Platform,I don't typically s...","Mercurial,Subversion,Other",Always,NaN,NaN,I am not currently employed
3,4,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",Yes,Operations Research Practitioner,Poorly,Self-employed,TensorFlow,...,100% of projects,Entirely internal,Standalone Team,"Column-oriented relational (e.g. KDB/MariaDB),...","Company Developed Platform,Email",Generic cloud file sharing software (Dropbox/B...,NaN,250000.0,USD,Has increased 20% or more
4,5,Male,Taiwan,38.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by a company that doesn't perform adv...,TensorFlow,...,10-25% of projects,Approximately half internal and half external,Business Department,Flat files not in a database or cache (e.g. CS...,Company Developed Platform,Git,Rarely,NaN,NaN,I do not want to share information about my sa...
5,6,Male,Brazil,46.0,Employed full-time,Yes,Data Scientist,Fine,Employed by college or university,TensorFlow,...,76-99% of projects,More internal than external,Other,"Column-oriented relational (e.g. KDB/MariaDB),...",I don't typically share data,"Bitbucket,Git,Other",Sometimes,NaN,NaN,I do not want to share information about my sa...
6,7,Male,United States,35.0,Employed full-time,Yes,Computer Scientist,Fine,Employed by government,TensorFlow,...,26-50% of projects,Entirely internal,IT Department,Flat files not in a database or cache (e.g. CS...,"Company Developed Platform,Email","Bitbucket,Git",NaN,NaN,NaN,NaN
8,9,Female,Australia,43.0,Employed full-time,Yes,Business Analyst,Fine,Employed by non-profit or NGO,Microsoft Excel Data Mining,...,Less than 10% of projects,Entirely internal,IT Department,Row-oriented relational (e.g. MySQL/Microsoft ...,"I don't typically share data,Share Drive/Share...",Generic non-cloud file sharing software (Email...,Rarely,80000.0,AUD,Has stayed about the same (has not increased o...
9,10,Male,Russia,33.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by a company that doesn't perform adv...,C/C++,...,Less than 10% of projects,Do not know,IT Department,Document-oriented (e.g. MongoDB/Elasticsearch)...,I don't typically share data,"Bitbucket,Git",Rarely,1200000.0,RUB,Has increased between 6% and 19%
11,12,Male,India,27.0,Employed full-time,Yes,Data Scientist,Fine,Employed by professional services/consulting f...,Other,...,26-50% of projects,Entirely internal,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,Company Developed Platform,Bitbucket,Rarely,95000.0,INR,Has increased 20% or more
14,15,Male,Taiwan,26.0,Employed full-time,Yes,Software Developer/Software Engineer,Fine,Employed by professional services/consulting firm,TensorFlow,...,26-50% of projects,Do not know,IT Department,Flat files not in a database or cache (e.g. CS...,"Email,Share Drive/SharePoint",Git,Sometimes,1100000.0,TWD,Has stayed about the same (has not increased o...
15,16,Male,United States,58.0,"Independent contractor, freelancer, or self-em...",Yes,DBA/Database Engineer,Poorly,Employed by professional services/consulting firm,Python,...,26-50% of projects,Do not know,Standalone Team,Document-oriented (e.g. MongoDB/Elasticsearch)...,"Company Developed Platform,Share Drive/SharePoint","Git,Subversion",Most of the time,120000.0,NaN,Has decreased between 6% and 19%


In [87]:
imp_freq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_imputed = imp_freq.fit_transform(X)

In [88]:
X_imputed

array([[1, 'Non-binary, genderqueer, or gender non-conforming',
        'United States', ..., 100000.0, 'USD',
        'I am not currently employed'],
       [4, 'Male', 'United States', ..., 250000.0, 'USD',
        'Has increased 20% or more'],
       [5, 'Male', 'Taiwan', ..., 100000.0, 'USD',
        'I do not want to share information about my salary/compensation'],
       ..., 
       [16688, 'Male', 'Malaysia', ..., 100000.0, 'USD',
        'I do not want to share information about my salary/compensation'],
       [16701, 'Male', 'South Korea', ..., 10000.0, 'USD',
        'I am not currently employed'],
       [16713, 'Male', 'Indonesia', ..., 100000.0, 'IDR',
        'I do not want to share information about my salary/compensation']], dtype=object)

In [89]:
enc = OneHotEncoder(handle_unknown= 'ignore')
X_imputed= enc.fit_transform(X_imputed)


In [90]:
X, X_test, y, y_test = train_test_split(X_imputed, y, test_size=.2, random_state=100)


In [117]:
clf = RandomForestClassifier(max_depth=10, n_estimators=500 ,random_state=7)
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=7, verbose=0, warm_start=False)

In [118]:
clf.score(X_test, y_test)

0.60778443113772451

In [119]:
confusion_matrix(y_test, clf.predict(X_test))

array([[  1, 524],
       [  0, 811]])

In [116]:
Counter(list(clf.feature_importances_))

Counter({0.0: 29723,
         0.00023414942195196017: 1,
         0.00012494174777381809: 1,
         8.8887630637420449e-05: 1,
         0.00018854688584529661: 1,
         3.6887783758683051e-07: 1,
         1.545042656774975e-05: 1,
         1.115686221568771e-06: 1,
         6.1481426333619669e-05: 1,
         0.00024563409382617643: 1,
         0.00038122754446152782: 1,
         0.00038493258679457999: 1,
         0.00019508215492781709: 1,
         2.1972924958924455e-05: 1,
         4.9172837172849212e-06: 1,
         0.0002243056941651892: 1,
         0.00010087452559664852: 1,
         1.8665218581895763e-05: 2,
         1.0307075360589517e-05: 1,
         8.4623192195202752e-05: 1,
         0.00018487543296885422: 1,
         0.00010273911391257408: 1,
         7.3228584253397086e-06: 1,
         0.00010082879770222166: 1,
         6.8332997422832718e-05: 1,
         0.00012707304083474195: 1,
         0.00016273441194461629: 1,
         0.00036635975944785136: 1,
         0

In [95]:
Counter(list(clf.predict(X_test)))

Counter({'1': 1336})

In [96]:
knn = KNeighborsClassifier(n_neighbors=25)
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=25, p=2,
           weights='uniform')

In [97]:
confusion_matrix(y_test, knn.predict(X_test))

array([[142, 383],
       [108, 703]])

In [98]:
knn.score(X_test,y_test)

0.63248502994011979

In [99]:
gbm = GradientBoostingClassifier()
gbm.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [100]:
gbm.score(X_test,y_test)

0.65194610778443118

In [101]:
confusion_matrix(y_test, gbm.predict(X_test))

array([[156, 369],
       [ 96, 715]])

In [102]:
adaboost = AdaBoostClassifier()
adaboost.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [103]:
confusion_matrix(y_test, adaboost.predict(X_test))

array([[218, 307],
       [137, 674]])

In [104]:
adaboost.score(X_test,y_test)

0.66766467065868262

In [105]:
linearsvc = LinearSVC()
linearsvc.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [106]:
linearsvc.score(X_test,y_test)

0.6272455089820359

In [107]:
svc = SVC(gamma='scale',probability=True)
svc.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [108]:
svc.score(X_test,y_test)

0.60928143712574845

In [109]:
confusion_matrix(y_test, svc.predict(X_test))

array([[  3, 522],
       [  0, 811]])